# Convert YoloV5 to ONNX on SageMaker Notebook Instance

The official document is [here](https://github.com/ultralytics/yolov5/issues/251). Here is demostration how to use SageMaker jupyter notebook to export YoloV5 model to ONNX.

In [ ]:
!pip install tensorboard onnx torch torchvision --upgrade

In [ ]:
# if loaded yolov5
# !rm -rf yolov5

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [1]:
# Add yolov5 system path
import sys, os, onnx, torch

In [2]:
print('\nStarting TorchScript export with torch %s...' % torch.__version__)


Starting TorchScript export with torch 1.5.1...


In [3]:
fp_yolov5 = os.path.abspath(os.getcwd()) + "/yolov5"
sys.path.append(fp_yolov5)
sys.path

['',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python36.zip',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/lib-dynload',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/extensions',
 '/home/ec2-user/.ipython',
 '/home/ec2-user/SageMaker/myAWSStudyBlog/sagemaker-greengrass/0-yolov5-onnx/yolov5']

### Change working directory to yolov5

In [4]:
%cd ./yolov5

/home/ec2-user/SageMaker/myAWSStudyBlog/sagemaker-greengrass/0-yolov5-onnx/yolov5


# Download pre-trained models

In [5]:
!python -c "from utils.google_utils import *; attempt_download('weights/yolov5s.pt');"

In [6]:
!ls weights

download_weights.sh  yolov5s.pt


# Export a Trained YoloV5 Model

In [ ]:
# !python models/export.py --weights yolov5s.pt --img 640 --batch 1 

In [7]:
img = torch.zeros((1, 3, 640, 640))

In [8]:
model = torch.load('./weights/yolov5s.pt', map_location=torch.device('cpu'))['model'].float()
model.eval()
# set Detect() layer export
model.model[-1].export = True
y = model(img)  # dry run

In [9]:
fp_torchscript = './yolov5s.torchscript.pt'  # filename
ts = torch.jit.trace(model, img)
ts.save(fp_torchscript)

In [10]:
fp_onnx = './yolov5s-detect.onnx'  # filename
# set Detect() layer export=False, ref [issue343](https://github.com/ultralytics/yolov5/issues/343#issuecomment-658021043) 
model.fuse()  # only for ONNX
torch.onnx.export(model, img, fp_onnx, verbose=False, opset_version=12, input_names=['images'],
                          output_names=['classes', 'boxes'] if y is None else ['output'])

Fusing layers... Model Summary: 140 layers, 7.45958e+06 parameters, 7.45958e+06 gradients


In [11]:
fp_onnx = './yolov5s.onnx'  # filename

model = torch.load('./weights/yolov5s.pt', map_location=torch.device('cpu'))['model'].float()
model.eval()
# ref [issue343](https://github.com/ultralytics/yolov5/issues/343#issuecomment-658021043)
model.model[-1].export = False
y = model(img)  # dry run
model.fuse()  # only for ONNX
torch.onnx.export(model, img, fp_onnx, verbose=False, opset_version=12, input_names=['images'],
                          output_names=['classes', 'boxes'] if y is None else ['output'])

Fusing layers... Model Summary: 140 layers, 7.45958e+06 parameters, 7.45958e+06 gradients


/home/ec2-user/SageMaker/myAWSStudyBlog/sagemaker-greengrass/0-yolov5-onnx/yolov5/models/yolo.py:30: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4]:


In [12]:
onnx_model = onnx.load(fp_onnx)  # load onnx model
onnx.checker.check_model(onnx_model)  # check onnx model
print(onnx.helper.printable_graph(onnx_model.graph))  # print a human readable model
print('ONNX export success, saved as %s' % fp_onnx)

graph torch-jit-export (
  %images[FLOAT, 1x3x640x640]
) initializers (
  %model.0.conv.conv.bias[FLOAT, 32]
  %model.0.conv.conv.weight[FLOAT, 32x12x3x3]
  %model.1.conv.bias[FLOAT, 64]
  %model.1.conv.weight[FLOAT, 64x32x3x3]
  %model.10.conv.bias[FLOAT, 256]
  %model.10.conv.weight[FLOAT, 256x512x1x1]
  %model.13.bn.bias[FLOAT, 256]
  %model.13.bn.num_batches_tracked[INT64, scalar]
  %model.13.bn.running_mean[FLOAT, 256]
  %model.13.bn.running_var[FLOAT, 256]
  %model.13.bn.weight[FLOAT, 256]
  %model.13.cv1.conv.bias[FLOAT, 128]
  %model.13.cv1.conv.weight[FLOAT, 128x512x1x1]
  %model.13.cv2.weight[FLOAT, 128x512x1x1]
  %model.13.cv3.weight[FLOAT, 128x128x1x1]
  %model.13.cv4.conv.bias[FLOAT, 256]
  %model.13.cv4.conv.weight[FLOAT, 256x256x1x1]
  %model.13.m.0.cv1.conv.bias[FLOAT, 128]
  %model.13.m.0.cv1.conv.weight[FLOAT, 128x128x1x1]
  %model.13.m.0.cv2.conv.bias[FLOAT, 128]
  %model.13.m.0.cv2.conv.weight[FLOAT, 128x128x3x3]
  %model.14.conv.bias[FLOAT, 128]
  %model.14.conv.we

### Move tourch script and ONNX model out

In [ ]:
!cp {fp_torchscript} ../yolov5s.torchscript.pt
!cp {fp_onnx} ../yolov5s.onnx

In [ ]:
# use https://lutzroeder.github.io/netron/ to check the model